# Impresión automatizada 

In [ ]:
import imaplib
import email
from email.header import decode_header
import os
import subprocess
from tempfile import NamedTemporaryFile

# Función para conectar al servidor de correo
def connect_to_email(username, password, imap_url):
    mail = imaplib.IMAP4_SSL(imap_url)
    mail.login(username, password)
    mail.select("inbox")
    return mail

# Función para listar correos electrónicos
def list_emails(mail, num_emails=10):
    status, data = mail.search(None, 'ALL')
    email_ids = data[0].split()
    email_ids = email_ids[-num_emails:]  # Obtener los últimos 'num_emails' correos

    email_list = []
    for email_id in email_ids:
        status, msg_data = mail.fetch(email_id, '(RFC822)')
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])
                subject, encoding = decode_header(msg["Subject"])[0]
                if isinstance(subject, bytes):
                    subject = subject.decode(encoding if encoding else 'utf-8')
                from_ = msg.get("From")
                email_list.append((email_id.decode(), subject, from_))

    return email_list

# Función para imprimir archivos adjuntos directamente
def print_attachments_directly(mail, email_id, printer_name):
    try:
        status, data = mail.fetch(email_id, "(RFC822)")
    except imaplib.IMAP4.abort:
        print("Error: La conexión fue abortada. Intentando reconectar...")
        mail = connect_to_email(username, password, imap_url)
        status, data = mail.fetch(email_id, "(RFC822)")

    for response_part in data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            for part in msg.walk():
                if part.get_content_maintype() == "multipart":
                    continue
                if part.get("Content-Disposition") is None:
                    continue
                filename = part.get_filename()
                if filename:
                    decoded_filename, encoding = decode_header(filename)[0]
                    if isinstance(decoded_filename, bytes):
                        decoded_filename = decoded_filename.decode(encoding if encoding else 'utf-8')
                    
                    with NamedTemporaryFile(delete=False, suffix=os.path.splitext(decoded_filename)[1]) as temp_file:
                        temp_file.write(part.get_payload(decode=True))
                        temp_file.flush()
                        temp_file_path = temp_file.name

                    # Usar Adobe Acrobat Reader para imprimir el archivo PDF si es un PDF
                    if temp_file_path.lower().endswith('.pdf'):
                        subprocess.run([
                            "C:\\Program Files (x86)\\Adobe\\Reader 9.0\\Reader\\AcroRd32.exe",
                            "/t", temp_file_path, printer_name
                        ])
                    else:
                        print(f"El archivo {decoded_filename} no es un PDF y no se puede imprimir con Acrobat Reader.")
                    
                    print(f"Sent {decoded_filename} to the printer {printer_name}")
                    os.remove(temp_file_path)  # Eliminar el archivo temporal después de imprimir

# Configuración de la cuenta de correo
username = "papeleriabunny1@gmail.com"
password = "jylk nvgr vxxm hddn"
imap_url = "imap.gmail.com"

# Conectarse al servidor de correo
mail = connect_to_email(username, password, imap_url)

# Listar los últimos correos electrónicos
emails = list_emails(mail, num_emails=10)

# Imprimir los IDs de los correos electrónicos
print("Últimos correos electrónicos:")
for email_id, subject, from_ in emails:
    print(f"ID: {email_id}, Asunto: {subject}, De: {from_}")

# Pedir al usuario que elija el correo para imprimir los adjuntos
selected_email_id = input("Introduce el ID del correo del que deseas imprimir los adjuntos: ")

# Imprimir adjuntos del correo electrónico específico
print_attachments_directly(mail, selected_email_id, "BLANCO Y NEGRO")